In [3]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

In [4]:
h2o.init(nthreads=-1, max_mem_size=9)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,13 mins 00 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,"28 days, 21 hours and 58 minutes"
H2O_cluster_name:,H2O_from_python_mujin_3zxcwy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,8 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [28]:
# df = pd.read_csv("time_data/ASOS_TIM.csv", encoding='cp949')
# print(type(df))

data = "https://raw.githubusercontent.com/Nyscrodia/WeatherForecasting/master/time_data/ASOS_TIM.csv"
df = h2o.import_file(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [29]:
df.head()

spot,spotName,date,Ta(筌욏뇭),Rn(mm),Ws(m/s),Wd(16D),Hm(%),Pa(hPa),Ps(hPa),Ss(hr),Gsr(MJ/m2),Snow(cm),DpthFhsc(cm),Tca(10),CloudForm,Ts(筌욏뇭)
nan,spotName,date,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,CloudForm,nan
108,Seoul,2021-01-01 0:00,-8.3,nan,1.8,250,66,1016.4,1027.7,nan,nan,nan,nan,0,,-6.8
108,Seoul,2021-01-01 1:00,-8.7,nan,2.4,270,68,1016.4,1027.7,nan,nan,nan,nan,0,,-6.9
108,Seoul,2021-01-01 2:00,-9.1,nan,1.6,270,69,1016.2,1027.5,nan,nan,nan,nan,0,,-7.1
108,Seoul,2021-01-01 3:00,-9.3,nan,1.1,250,70,1016.8,1028.1,nan,nan,nan,nan,0,,-7.3
108,Seoul,2021-01-01 4:00,-9.3,nan,0.3,0,71,1016.2,1027.5,nan,nan,nan,nan,0,,-7.5
108,Seoul,2021-01-01 5:00,-9.7,nan,1.9,20,72,1015.6,1026.9,nan,nan,nan,nan,0,,-7.6
108,Seoul,2021-01-01 6:00,-9.7,nan,2,50,75,1015.4,1026.7,nan,nan,nan,nan,7,Sc,-7.4
108,Seoul,2021-01-01 7:00,-9.3,nan,1.6,50,71,1015.6,1026.9,nan,nan,nan,nan,6,Sc,-6.8
108,Seoul,2021-01-01 8:00,-9.3,nan,1.6,50,72,1015.8,1027.1,0,0,nan,nan,1,Sc,-7.3


In [31]:
df["Rn(mm)"] = df['Rn(mm)'].asfactor()
print(df['Rn(mm)'].levels())

H2OValueError: Only 'int' or 'string' are allowed for asfactor(), got Rn(mm):real 

In [16]:
df.describe()

,spot,Ta(°C),Rn(mm),Ws(m/s),Wd(16D),Hm(%),Pa(hPa),Ps(hPa),Ss(hr),Gsr(MJ/m2),Snow(cm),DpthFhsc(cm),Tca(10),Ts(°C)
count,8760.0,8760.000000,949.000000,8758.000000,8758.000000,8760.000000,8760.000000,8760.000000,4791.000000,4791.000000,379.000000,29.000000,8758.000000,8755.000000
mean,108.0,13.736678,1.250263,2.311270,169.667732,65.589269,1006.047785,1016.281621,0.458380,1.076706,1.397625,0.962069,4.959922,14.731845
std,0.0,10.896708,3.398801,1.120855,107.546970,18.393227,7.928486,8.312023,0.438788,0.966440,1.357390,1.424132,3.929389,12.043577
min,108.0,-18.500000,0.000000,0.000000,0.000000,19.000000,986.900000,996.500000,0.000000,0.000000,0.000000,0.000000,0.000000,-12.700000
25%,108.0,6.000000,0.000000,1.500000,50.000000,51.000000,999.800000,1009.700000,0.000000,0.230000,0.400000,0.000000,0.000000,4.300000
50%,108.0,14.600000,0.100000,2.200000,200.000000,66.000000,1006.300000,1016.400000,0.400000,0.790000,0.900000,0.200000,6.000000,15.400000
75%,108.0,22.600000,1.100000,2.900000,270.000000,80.000000,1012.000000,1022.500000,1.000000,1.800000,1.850000,0.900000,9.000000,23.600000
max,108.0,36.300000,64.700000,8.300000,360.000000,100.000000,1025.800000,1037.400000,1.000000,3.740000,5.100000,4.400000,10.000000,56.800000


In [26]:
feature = df.drop('Rn(mm)', axis=1)
feature.describe()
print(type(feature))

<class 'pandas.core.frame.DataFrame'>


In [27]:
label = df['Rn(mm)']
label.describe()
print(type(label))

<class 'pandas.core.series.Series'>


In [24]:
label.to_frame(["Rn(mm)"])
#label = label.asfactor()

TypeError: unhashable type: 'list'

TypeError: unhashable type: 'list'